# 🧼 Veefyed Web Scraping & Ingredient Matching Task

This notebook demonstrates a two-part task:
1. **Scraping product data** from [snapklik.com](https://snapklik.com)
2. **Grouping skincare products** by shared ingredients.

We use Python tools including `requests`, `BeautifulSoup`, `pandas`, and `re` for this task.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import uuid

BASE_URL = "https://snapklik.com"
CATEGORY_URL = f"{BASE_URL}/collections/skincare"
headers = { "User-Agent": "Mozilla/5.0" }

def get_product_links():
    response = requests.get(CATEGORY_URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    product_links = []
    for a in soup.select("a.full-unstyled-link"):
        href = a.get("href")
        if href and href.startswith("/products"):
            full_url = BASE_URL + href
            if full_url not in product_links:
                product_links.append(full_url)
        if len(product_links) >= 10:
            break
    return product_links

def scrape_product(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    def safe_text(selector):
        tag = soup.select_one(selector)
        return tag.get_text(strip=True) if tag else "Nill"

    product_id = str(uuid.uuid5(uuid.NAMESPACE_URL, url))
    product_name = safe_text("h1.product-title")
    brand_name = "Nill"
    product_description = safe_text(".product__description")
    price = safe_text(".price")
    image_tag = soup.select_one(".product__media img")
    product_image = image_tag['src'] if image_tag else "Nill"
    ingredients = "Nill"

    if "ingredient" in product_description.lower():
        match = re.findall(r"ingredients?:\s*(.+?)(?:\.|\n|$)", product_description, re.IGNORECASE)
        if match:
            ingredients = match[0]

    size = re.search(r"\b(\d+\s*(ml|g|oz))\b", product_description, re.IGNORECASE)
    size_volume = size.group(1) if size else "Nill"

    return {
        "Product ID": product_id,
        "Product Line Name": "Nill",
        "Brand Name": brand_name,
        "Product Name": product_name,
        "Product Description": product_description,
        "Product Images": BASE_URL + product_image if product_image.startswith("/") else product_image,
        "Barcode (EAN/UPC)": "Nill",
        "Price": price,
        "Size/Volume": size_volume,
        "Ingredients": ingredients,
        "Skin Concern": "Nill",
        "Source URL": url
    }

# Get links and scrape
links = get_product_links()
products = [scrape_product(link) for link in links]
df = pd.DataFrame(products)
df.to_csv("veefyed_product_data.csv", index=False)
df.head()


In [ ]:
def preprocess_ingredients(ing_str):
    return set(re.sub(r"[^a-zA-Z,]", "", ing_str.lower()).split(",")) if ing_str != "Nill" else set()

grouped = []
group_id = "A"
already_grouped = set()

for i in range(len(df)):
    if i in already_grouped:
        continue
    ing_i = preprocess_ingredients(df.loc[i, "Ingredients"])
    group = [df.loc[i, "Product Name"]]
    shared_ings = ing_i.copy()
    for j in range(i+1, len(df)):
        ing_j = preprocess_ingredients(df.loc[j, "Ingredients"])
        if len(ing_i & ing_j) >= 2:
            group.append(df.loc[j, "Product Name"])
            shared_ings &= ing_j
            already_grouped.add(j)
    if len(group) > 1:
        grouped.append({
            "Group": group_id,
            "Shared Ingredients": ", ".join(shared_ings),
            "Product Names": ", ".join(group)
        })
        group_id = chr(ord(group_id) + 1)

group_df = pd.DataFrame(grouped)
group_df.to_excel("veefyed_ingredient_groups.xlsx", index=False)
group_df


### ✅ Task Summary

- Scraped skincare product data from Snapklik
- Extracted core features including ingredients
- Grouped products with 2+ overlapping ingredients
- Saved output to Excel and CSV files

This notebook can be extended further using Selenium for dynamic content or enhanced NLP for better ingredient parsing.
